# Análisis de los datos covid con pandas

Conocimientos previos de Python recomendables:

  - [Booleanos](../../python/booleanos.ipynb)
  - [Control de flujo](../../python/control_de_flujo.ipynb)
  - [Bloques e indentación](../../python/bloques_e_indentacion.ipynb)
  - [Listas y secuencias](../../python/secuencias.ipynb)
  - [Diccionarios](../../python/diccionarios.ipynb)
  - [Modulos](../../python/modulos.ipynb)
  - [funciones](../../python/funciones.ipynb)
  - [variables](../../python/tipos_y_variables.ipynb)
  
Conocimientos previos de análisis de datos en Python recomendables:

  - [Herramientas](../../analisis/herramientas.ipynb)
  - [Numpy](../../analisis/numpy.ipynb)
  - [Pandas](../../analisis/pandas.ipynb)
  - [Matplotlib](../../analisis/matplotlib.ipynb)

# Lectura del fichero


In [ ]:
from pathlib import Path
import pandas

covid_csv_path = Path('..') / 'downloaded_data' / 'casos_hosp_uci_def_sexo_edad_provres.csv'
dframe = pandas.read_csv(covid_csv_path, delimiter=',', parse_dates=['fecha'], index_col='fecha')
print(dframe.shape)
dframe

# Evolución a lo largo del tiempo

Calcula como han variado a lo largo del tiempo el número de casos, hospitalizaciones, ingresos en UCI y defunciones.

In [ ]:
dframe_por_fecha = dframe.groupby(by='fecha').sum()
dframe_por_fecha

# Gráficas con pandas

Dibuja, utilizando pandas, los datos anteriores.

In [ ]:
dframe_por_fecha.plot(y='num_casos')

In [ ]:
dframe_por_fecha.plot(y='num_def')

# Gráficas con matplotlib

In [ ]:
from matplotlib import pyplot as plt

plt.plot(dframe_por_fecha.index, dframe_por_fecha['num_casos'])

# Suma de datos semanales

Los datos diarios tienen bastante ruido debido a distintos factores. Obtén los datos semanales para todos los parámetros.

In [ ]:
dframe_semanal = dframe.resample("7D").sum()
dframe_semanal.plot(y='num_casos')

# Selección de una ola

Calcula los datos para una sola ola.

Como orientación podrías utilizar los siguientes límites entre olas.

In [ ]:
from datetime import datetime
WAVE_LIMITS = [
    datetime(2020, 2, 1),
    datetime(2020, 7, 15),
    datetime(2020, 10, 11),
    datetime(2020, 12, 20),
    datetime(2021, 6, 18),
    datetime(2021, 10, 31),
    datetime(2022, 3, 10),
]

In [ ]:
from datetime import datetime
date0 = datetime(year=2020, month=12, day=1)
date1 = datetime(year=2021, month=3, day=1)
# seleccionamos indexando por fecha
tercera_ola = dframe.loc[date0:date1, :]
tercera_ola.resample('7D').sum().plot(y='num_casos')

# Análisis de una provincia

Determina cuáles son las provincias presentes en el DataFrame y dibuja los datos por provincia.

In [ ]:
print(dframe['provincia_iso'].unique())
#filtramos por provincia
valencia = dframe[dframe['provincia_iso']=='V']
valencia = valencia.resample('7D').sum()
valencia.plot(y='num_def')

# Selección de un sexo o grupo de edad

In [ ]:
print(dframe['grupo_edad'].unique())
ninyos = dframe[dframe['grupo_edad']=='0-9']
ninyos = valencia.resample('7D').sum()
ninyos.plot(y='num_hosp')

# Comparación por provincias

In [ ]:
casos_por_provincia = dframe.groupby(by='provincia_iso').sum()
casos_por_provincia['num_def'].plot.bar()

# Comparación por olas

In [ ]:
limites_entre_olas = [
    datetime(2020, 7, 15),
    datetime(2020, 10, 11),
    datetime(2020, 12, 20),
    datetime(2021, 6, 18),
    datetime(2021, 10, 31),
    datetime.now(),
]

def calcular_num_ola_dada_fecha(fecha):
    for idx, limite in enumerate(limites_entre_olas):
        if fecha <= limite:
            return idx + 1

num_ola_para_fechas = [calcular_num_ola_dada_fecha(fecha) for fecha in dframe.index]
datos_por_olas = dframe.groupby(by=num_ola_para_fechas).sum()
axes_suplot = datos_por_olas['num_hosp'].plot.bar()
axes_suplot.set_ylabel('Num. hospitalizaciones')
axes_suplot.set_xlabel('ola')
